In [313]:
import pandas as pd
import numpy as np

def main(cgm,insulin):
   #Read CGM Data
    cgmdata=pd.read_csv(cgm)
    #Read Insulin Data
    insulinData=pd.read_csv(insulin)
     
    #Timestamp indentification for Auto Mode and Manual Mode
    insulinData[insulinData["Alarm"]=='AUTO MODE ACTIVE PLGM OFF']
    Date=insulinData["Date"].iloc[-1]
    Time=insulinData["Time"].iloc[-1]
    
    #subset cgm to required column
    cgmdata=cgmdata[["Date","Time","Sensor Glucose (mg/dL)"]]
    
    #Imput missing observations with mean value
    cgmdata['Sensor Glucose (mg/dL)'].fillna(cgmdata['Sensor Glucose (mg/dL)'].mean(), inplace = True)
   
    #Estimate Start for Auto Mode and manual Mode on CGM data
    cgmdata['ManualMode']=cgmdata.apply(lambda row:row.Date>Date and row.Time>Time,axis=1)
    
    ########### DAY METRICS ############
    cgmdata['CGM >180mg']= np.where(cgmdata['Sensor Glucose (mg/dL)'] > 180, '>180 mg','false')
    cgmdata['CGM >250mg']= np.where(cgmdata['Sensor Glucose (mg/dL)'] > 250, '>250 mg','false')
    cgmdata['70mg<=CGM<=180mg'] = np.where(cgmdata['Sensor Glucose (mg/dL)'].between(70,180) , '70mg<=CGM<=180mg', 'false')
    cgmdata['70mg<=CGM<=150mg'] = np.where(cgmdata['Sensor Glucose (mg/dL)'].between(70,150) , '70mg<=CGM<=150mg', 'false')
    cgmdata['CGM <70mg']= np.where(cgmdata['Sensor Glucose (mg/dL)'] < 70, '<70 mg','false')
    cgmdata['CGM <54mg']= np.where(cgmdata['Sensor Glucose (mg/dL)'] < 54, '<54 mg','false')
    
    ####### Percentage of time >180mg #########
    cgmday_1= cgmdata[cgmdata['CGM >180mg'] != 'false']
    cgmday1 = cgmday_1.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count1 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday11=cgmday1.div(cgm_count1, level='ManualMode') * 100
    
    cgmday11.reset_index(level=0, inplace=True)
    
     ####### Percentage of time >250mg #########
    cgmday_2= cgmdata[cgmdata['CGM >250mg'] != 'false']
    cgmday2 = cgmday_2.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count2 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday12=cgmday2.div(cgm_count1, level='ManualMode') * 100
    
    cgmday12.reset_index(level=0, inplace=True)
    
     ####### Percentage of time 70mg<=CGM<=180mg #########
    cgmday_3= cgmdata[cgmdata['70mg<=CGM<=180mg'] != 'false']
    cgmday3 = cgmday_3.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count3 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday13=cgmday3.div(cgm_count1, level='ManualMode') * 100
    
    cgmday13.reset_index(level=0, inplace=True)
    
    
     ####### Percentage of time 70mg<=CGM<=150mg #########
    cgmday_4= cgmdata[cgmdata['70mg<=CGM<=150mg'] != 'false']
    cgmday4 = cgmday_4.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count4 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday14=cgmday4.div(cgm_count1, level='ManualMode') * 100
    
    cgmday14.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <70 #########
    cgmday_5= cgmdata[cgmdata['CGM <70mg'] != 'false']
    cgmday5 = cgmday_5.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count5 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday15=cgmday5.div(cgm_count1, level='ManualMode') * 100
    
    cgmday15.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <54 #########
    cgmday_6= cgmdata[cgmdata['CGM <54mg'] != 'false']
    cgmday6 = cgmday_6.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count6 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday16=cgmday6.div(cgm_count1, level='ManualMode') * 100
    
    cgmday16.reset_index(level=0, inplace=True)
    
    ## Merge Matrix
    merged_inner = pd.merge(left=cgmday11, right=cgmday12, left_on='ManualMode', right_on='ManualMode')
    merged_inner1 = pd.merge(left=merged_inner, right=cgmday13, left_on='ManualMode', right_on='ManualMode')
    merged_inner2 = pd.merge(left=merged_inner1, right=cgmday14, left_on='ManualMode', right_on='ManualMode')
    merged_inner3 = pd.merge(left=merged_inner2, right=cgmday15, left_on='ManualMode', right_on='ManualMode')
    merged_final = pd.merge(left=merged_inner3, right=cgmday16, left_on='ManualMode', right_on='ManualMode')
    
    
    ####Daytime####
    cgmdata['hour'] = pd.to_datetime(cgmdata['Time'], format='%H:%M:%S').dt.hour
    cgmdata['Daytime'] = np.where(cgmdata['hour'].between(6,23) , 'Day', 'false')
    cgmdaytime= cgmdata[cgmdata['Daytime'] != 'false']
    
    
    ####### Percentage of time >180mg --- Daytime #########
    cgmday_1= cgmdaytime[cgmdata['CGM >180mg'] != 'false']
    cgmday1 = cgmday_1.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count1 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday11=cgmday1.div(cgm_count1, level='ManualMode') * 100
    
    cgmday11.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time >250mg--- Daytime #########
    cgmday_2= cgmdaytime[cgmdata['CGM >250mg'] != 'false']
    cgmday2 = cgmday_2.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count2 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday12=cgmday2.div(cgm_count1, level='ManualMode') * 100
    
    cgmday12.reset_index(level=0, inplace=True)
    
     ####### Percentage of time 70mg<=CGM<=180mg--- Daytime #########
    cgmday_3= cgmdaytime[cgmdata['70mg<=CGM<=180mg'] != 'false']
    cgmday3 = cgmday_3.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count3 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday13=cgmday3.div(cgm_count1, level='ManualMode') * 100
    
    cgmday13.reset_index(level=0, inplace=True)
    
    
     ####### Percentage of time 70mg<=CGM<=150mg--- Daytime #########
    cgmday_4= cgmdaytime[cgmdata['70mg<=CGM<=150mg'] != 'false']
    cgmday4 = cgmday_4.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count4 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday14=cgmday4.div(cgm_count1, level='ManualMode') * 100
    
    cgmday14.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <70--- Daytime #########
    cgmday_5= cgmdaytime[cgmdata['CGM <70mg'] != 'false']
    cgmday5 = cgmday_5.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count5 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday15=cgmday5.div(cgm_count1, level='ManualMode') * 100
    
    cgmday15.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <54--- Daytime #########
    cgmday_6= cgmdaytime[cgmdata['CGM <54mg'] != 'false']
    cgmday6 = cgmday_6.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count6 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday16=cgmday6.div(cgm_count1, level='ManualMode') * 100
    
    cgmday16.reset_index(level=0, inplace=True)
    
    ## Merge Matrix --- Daytime #########
    merged_inner = pd.merge(left=cgmday11, right=cgmday12, left_on='ManualMode', right_on='ManualMode')
    merged_inner1 = pd.merge(left=merged_inner, right=cgmday13, left_on='ManualMode', right_on='ManualMode')
    merged_inner2 = pd.merge(left=merged_inner1, right=cgmday14, left_on='ManualMode', right_on='ManualMode')
    merged_inner3 = pd.merge(left=merged_inner2, right=cgmday15, left_on='ManualMode', right_on='ManualMode')
    merged_finaldaytime = pd.merge(left=merged_inner3, right=cgmday16, left_on='ManualMode', right_on='ManualMode')
    
        ####Overnight####
    cgmovernight= cgmdata[cgmdata['Daytime'] == 'false']
    
    
    ####### Percentage of time >180mg --- Overnight #########
    cgmday_1= cgmovernight[cgmdata['CGM >180mg'] != 'false']
    cgmday1 = cgmday_1.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count1 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday11=cgmday1.div(cgm_count1, level='ManualMode') * 100
    
    cgmday11.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time >250mg--- Overnight #########
    cgmday_2= cgmovernight[cgmdata['CGM >250mg'] != 'false']
    cgmday2 = cgmday_2.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count2 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday12=cgmday2.div(cgm_count1, level='ManualMode') * 100
    
    cgmday12.reset_index(level=0, inplace=True)
    
     ####### Percentage of time 70mg<=CGM<=180mg--- Overnight#########
    cgmday_3= cgmovernight[cgmdata['70mg<=CGM<=180mg'] != 'false']
    cgmday3 = cgmday_3.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count3 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday13=cgmday3.div(cgm_count1, level='ManualMode') * 100
    
    cgmday13.reset_index(level=0, inplace=True)
    
    
     ####### Percentage of time 70mg<=CGM<=150mg--- Overnight #########
    cgmday_4= cgmovernight[cgmdata['70mg<=CGM<=150mg'] != 'false']
    cgmday4 = cgmday_4.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count4 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday14=cgmday4.div(cgm_count1, level='ManualMode') * 100
    
    cgmday14.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <70--- Overnight #########
    cgmday_5= cgmovernight[cgmdata['CGM <70mg'] != 'false']
    cgmday5 = cgmday_5.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count5 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday15=cgmday5.div(cgm_count1, level='ManualMode') * 100
    
    cgmday15.reset_index(level=0, inplace=True)
    
    
    ####### Percentage of time CGM <54--- Overnight #########
    cgmday_6= cgmovernight[cgmdata['CGM <54mg'] != 'false']
    cgmday6 = cgmday_6.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgm_count6 = cgmdata.groupby(['ManualMode']).agg({'Sensor Glucose (mg/dL)': 'count'})
    cgmday16=cgmday6.div(cgm_count1, level='ManualMode') * 100
    
    cgmday16.reset_index(level=0, inplace=True)
    
    ## Merge Matrix --- Overnight #########
    merged_inner = pd.merge(left=cgmday11, right=cgmday12, left_on='ManualMode', right_on='ManualMode')
    merged_inner1 = pd.merge(left=merged_inner, right=cgmday13, left_on='ManualMode', right_on='ManualMode')
    merged_inner2 = pd.merge(left=merged_inner1, right=cgmday14, left_on='ManualMode', right_on='ManualMode')
    merged_inner3 = pd.merge(left=merged_inner2, right=cgmday15, left_on='ManualMode', right_on='ManualMode')
    merged_finalovernight = pd.merge(left=merged_inner3, right=cgmday16, left_on='ManualMode', right_on='ManualMode')
    
    merged_all1 = pd.merge(left= merged_finalovernight, right=merged_finaldaytime, left_on='ManualMode', right_on='ManualMode')
    merged_all = pd.merge(left=merged_all1, right=merged_final, left_on='ManualMode', right_on='ManualMode')
   
    final_matrix=merged_all.sort_values(by=['ManualMode'],ascending=False)
    Results=final_matrix.drop(columns=['ManualMode'])
    
    ##output csv##
    return(Results.to_csv('Desktop/Project/Result.csv',header=False, index=False))
    
    

In [314]:
main('Desktop/Project/CGMData.csv','Desktop/Project/InsulinData.csv')

/Users/Akc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/Akc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/Akc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:98: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/Akc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:107: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/Akc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:115: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/Akc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:124: UserWarning: Boolean S